In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import common.common_machine_learning as common
import common.feature_num as feature_num
import common.features_precio_promedio_metroscubiertos as feature_metroscubiertos
import common.features_precio_promedio_ciudad as feature_ciudades

from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

import lightgbm as lgb

In [2]:
def evaluar_rf(modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    errors = abs(y_pred - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Performance del modelo:')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [3]:
TARGET = "precio"
MAX_ITER = 400

In [4]:
train = pd.read_csv('sets_de_datos/dima_train_with_features_6.csv', index_col = 0)
test = pd.read_csv('sets_de_datos/dima_test_with_features_6.csv', index_col = 0)

In [5]:
train.columns

Index(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       ...
       'ciudad_mean_antiguedad_sobre_provincia_mean_antiguedad',
       'tipodepropiead_mean_utilidades_extra_sobre_ciudad_mean_utilidades_extra',
       'antiguedad_sobre_tipodepropiedad_mean_antiguedad',
       'direccion_cantidad_al_menos_una_mayuscula',
       'direccion_cantidad_fijos_top_k', 'titulo_cantidad_fijos_top_k',
       'titulo_palabras_top_k_sobre_total_palabras',
       'ciudad_distancia_al_origen', 'ciudad_mean_mean_todas',
       'ciudad_mean_antiguedad_sobre_mean_metrocubiertos'],
      dtype='object', length=126)

In [6]:
columnas = [
'descripcion_longitud_media_de_palabra', 'metrostotales',
'metros_x_espacio', 'idzona', 'lat', 'titulo_longitud_media_de_palabra',
'distancia_minima_Oficina_comercial',
'distancia_minima_Local_en_centro_comercial',
'distancia_minima_Bodega_comercial', 'metros_x_garages',
'metroscubiertos', 'metros_x_habitaciones', 'metros_x_banos',
'distancia_minima_Terreno_comercial',
'distancia_minima_Local_Comercial',
'titulo_cantidad_caracteres_en_palabras_importantes',
'descripcion_cantidad_palabras_top_k',
'descripcion_cantidad_caracteres_en_palabras_importantes',
'coseno_distancia_angular_al_centro_provincia',
'descripcion_cantidad_stopwords',
'descripcion_cantidad_signos_puntacion',
'direccion_longitud_media_de_palabra',
'titulo_palabras_top_k_sobre_total_palabras',
'habitaciones_preciopromedio_ciudad', 'antiguedad', 'lng',
'garages_preciopromedio_ciudad',
'direccion_cantidad_caracteres_en_palabras_importantes',
'coseno_distancia_angular_al_centro_ciudad',
'banos_preciopromedio_ciudad', 'descripcion_cantidad_prefijos_top_k',
'precio_x_m2', 'año', 'titulo_cantidad_fijos_top_k',
'descripcion_cantidad_palabras_importantes',
'descripcion_cantidad_postfijos_top_k', 'ciudad_mean_mean_todas',
'ciudad_mean_antiguedad_sobre_mean_metrocubiertos',
'habitaciones_preciopromedio_metroscubiertos', 'cantidad_espacios',
'direccion_cantidad_al_menos_una_mayuscula',
'direccion_cantidad_fijos_top_k', 'ciudad_distancia_al_origen']

In [7]:
train = train[train["tipodepropiedad"].isin(test["tipodepropiedad"].unique())]

In [8]:
train['provincia'] = train['provincia'].str.replace("0    Distrito Federal","Distrito Federal")
test['provincia'] = test['provincia'].str.replace("0    Distrito Federal","Distrito Federal")

In [9]:
train = train[columnas + ["precio"]]
test = test[columnas]

In [10]:
#train_OHE  = pd.get_dummies(train)

In [11]:
#test_OHE = pd.get_dummies(test)

In [12]:
#train_OHE.shape

In [13]:
#test_OHE.shape

In [14]:
#X = train_OHE.drop([TARGET], axis = 1).copy().values
#y = list(train_OHE[TARGET].copy())

X = train.drop([TARGET], axis = 1).copy().values
y = list(train[TARGET].copy())

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 0)

In [16]:
hiperparametros = {
    "learning_rate": [0.03, 0.01, 0.1, 0.3],
    "n_estimators": [100, 200, 250, 300, 350, 400],
    "max_depth": [15, 20, 25, 30, 40, 50, 60],
    "reg_alpha": [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
    "reg_lambda": [0, 1e-1, 1, 5, 10, 20, 50, 100],
    "num_leaves": [5, 10, 15, 20, 25, 30],
    "min_child_weight": [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],

}

In [17]:
modelo = lgb.LGBMRegressor(learning_rate = 0.5)

inicio = time.time()


modelo_random = RandomizedSearchCV(estimator = modelo, param_distributions = hiperparametros, n_iter = MAX_ITER, cv = 3, verbose=2, random_state=42, n_jobs = -1)

fin = time.time()

modelo_random.fit(X_train, y_train)

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 25.5min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 45.5min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed: 73.0min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed: 88.7min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           importance_type='split',
                                           learning_rate=0.5, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=100,
                                           n_jobs=-1, num_leaves=31,
                                           objective=None, random_state=None,
                                           reg_alpha=0.0, reg_...
                                        'max_depth': [15, 20, 25, 30, 40, 50,
                                                      60],
                                        'mi

In [18]:
print("El tuneo tardó: {} minutos.".format((fin - inicio) / 60))

El tuneo tardó: 0.0 minutos.


In [19]:
print("Mejores parámetros:")
print(modelo_random.best_params_)

Mejores parámetros:
{'reg_lambda': 1, 'reg_alpha': 7, 'num_leaves': 30, 'n_estimators': 400, 'min_child_weight': 1000.0, 'max_depth': 50, 'learning_rate': 0.3}


In [20]:
mejores_hiperparametros = modelo_random.best_params_

In [21]:
mejor_lgb = lgb.LGBMRegressor(reg_lambda = mejores_hiperparametros["reg_lambda"],
                              num_leaves = mejores_hiperparametros["num_leaves"],
                              n_estimators = mejores_hiperparametros["n_estimators"],
                              min_child_weight = mejores_hiperparametros["min_child_weight"],
                              max_depth = mejores_hiperparametros["max_depth"],
                              learning_rate = mejores_hiperparametros["learning_rate"])
                                    
mejor_lgb.fit(X_train, y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.3, max_depth=50,
              min_child_samples=20, min_child_weight=1000.0, min_split_gain=0.0,
              n_estimators=400, n_jobs=-1, num_leaves=30, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=1, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [22]:
evaluar_rf(mejor_lgb, X_test, y_test)

Performance del modelo:
Average Error: 582670.1109 degrees.
Accuracy = 70.65%.


70.65210307356402

In [23]:
y_pred = mejor_lgb.predict(test)

In [24]:
df = pd.DataFrame({"id": test.index, "target": y_pred})

In [28]:
filename = 'LightLGB_06.csv'

df.to_csv(filename,index=False)